In [3]:
import pandas as pd
import numpy as np

In [4]:
df_data_metrics = pd.read_excel("INN_DNS_LB_Route_Min_Total_Metrics_2023-05-19.xlsx", sheet_name='data_metrics')

df_data_metrics_coppied = df_data_metrics.copy()

data_metrics_columns = ['00_AA_T3_1','00_AA_T3_2','00_BB_T3_4','00_BB_T3_5','01_CC_T3_1',	'01_CC_T3_2','06_DD_T3_1','06_DD_T3_2','06_EE_T3_6','06_EE_T3_8', '07_CC_T3_1','07_CC_T3_2','16_FF_T3_3','16_FF_T3_4','21_CC_T3_1'	'21_CC_T3_2','25_GG_T3_3','25_GG_T3_4','34_DD_T3_1','34_DD_T3_2',	'35_DD_T3_1','35_DD_T3_2','55_HH_T3_3','55_HH_T3_4']

In [10]:
class MyOptimizerAlgorithm:
    def __init__(self):

        self.the_metric = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.df_metric_score = pd.DataFrame(columns=['epoch','the_metric','score'])

    def start_optimizer(self, dataframe, min_vector, epoch, stop_score=None):
        self.df = dataframe
        self.min_vector = min_vector

        for epc in range(epoch):
            self.func1(epc)
            if (stop_score):
                if (self.score >= stop_score): 
                    print('STOPE_SCORE is supplied!')
                    print(f'Stope Score expectation: {stop_score}\nSupplied: {self.score}')
                    break
            if (epc == epoch-1): print('Epoch is done!')
                

                
        print("Here are the metric vectors that provide the highest scores achieved")
        print(f"The epoch to reach the maximum score first: {self.df_metric_score.sort_values(by=['score', 'epoch'], ascending=[False,True]).iloc[0,0]}")
        print('-'*20, "THE OPTIMIZER DATAFRAME", '-'*20)

        self.func_show_df(self.df_metric_score.sort_values(by=['score', 'epoch'], ascending=[False,True]))

    # step 1

    def func1(self, epoch):
        self.wanted_min_rows_current_order_vector = []
        for row in range(len(self.df.columns)):
            if self.df.iloc[:,row].idxmin() == self.min_vector[row]: # are current order and expected order of minimum equals?

                self.wanted_min_rows_current_order_vector.append(0)
            else: 
                wanted_min_rows_value = self.df.iloc[self.min_vector[row],row]
                row_sorted_series = self.df.iloc[:,row].sort_values().reset_index(drop=True)
                wanted_min_row_current_min_index = row_sorted_series[row_sorted_series == wanted_min_rows_value].index[0]
                self.wanted_min_rows_current_order_vector.append(wanted_min_row_current_min_index)

        df_smaller_analysis = self.func2(self.wanted_min_rows_current_order_vector) #self.df_func2_main is returned

        self.score = df_smaller_analysis['smaller_row_indexes'].isna().sum()
        self.func3(df_smaller_analysis) #self.df updated

        new_data = pd.DataFrame({
            'epoch': [epoch],
            'the_metric': [str(self.the_metric.tolist())],
            'score': [self.score]
        })
        self.df_metric_score = pd.concat([self.df_metric_score, new_data], ignore_index=True)



    # step 2

    def func2_2(self, df, column, wanted_min_index):
        df_2 = df.iloc[:, column].sort_values().reset_index().rename(columns = {'index': 'org_index'})
        indeX = df_2.loc[df_2['org_index'] == wanted_min_index].index[0]
        df_3 = df_2.iloc[:indeX,:]
        df_3.columns = ['smaller_row_indexes','smaller_row_values']
        return df_3

        

    def func2(self, min_row_index_vector):
        
        self.df_func2_main = pd.DataFrame(columns=['smaller_row_indexes','smaller_row_values', 'column_name'])

        for column in range(len(self.df.columns)):
            if min_row_index_vector[column] > 0:
                df_smaller_rows = self.func2_2(
                    df = self.df, 
                    column = column, 
                    wanted_min_index = self.min_vector[column])
            
                df_smaller_rows['min_addition_value'] = self.df.iloc[self.min_vector[column], column] - df_smaller_rows['smaller_row_values']
                df_smaller_rows['column_name'] = self.df.columns.to_list()[column]
                
                self.df_func2_main = pd.concat([self.df_func2_main, df_smaller_rows], axis = 0)
            
            elif min_row_index_vector[column] == 0:
                dic = {
                    'smaller_row_indexes': None,
                    'smaller_row_values': None,
                    'column_name': self.df.columns.to_list()[column]                  
                }
                df_np = pd.DataFrame(data = [dic])
                self.df_func2_main = pd.concat([self.df_func2_main, df_np], axis = 0)
        
        return self.df_func2_main

    def func3(self, df_smaller_analysis):
        df_smaller_analysis = df_smaller_analysis.reset_index(drop=True)
        df_group_count_smaller_rows = df_smaller_analysis.groupby(by=['smaller_row_indexes'], as_index=False).size()
        
        maks_size_row = df_group_count_smaller_rows['smaller_row_indexes'][df_group_count_smaller_rows['size'] == df_group_count_smaller_rows['size'].max()].iloc[0]
        
        df_smaller_analysis_max_size_row = df_smaller_analysis[df_smaller_analysis['smaller_row_indexes'] == maks_size_row]
        
        df_sa_optimum_addition = df_smaller_analysis_max_size_row[df_smaller_analysis_max_size_row['min_addition_value'] == df_smaller_analysis_max_size_row['min_addition_value'].max()].loc[:,['smaller_row_indexes', 'min_addition_value']]
        
        self.the_metric[df_sa_optimum_addition.iloc[0,0]] = self.the_metric[df_sa_optimum_addition.iloc[0,0]] + df_sa_optimum_addition.iloc[0,1] + 1
        
        self.addition_serie = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.addition_serie[df_sa_optimum_addition.iloc[0,0]] = df_sa_optimum_addition.iloc[0,1] + 1
        
        self.df = self.df.add(self.addition_serie, axis='index')

    
    def func_show_df(self, df):
        pd.set_option('display.max_colwidth', None)
        print(df.head(20))
        



In [6]:
expected_min_vector = [1,1,2,0,3,3,5,5,5,4,6,6,7,7,8,8,8,8,9,9,10,10,4,4]

In [11]:
algorithm = MyOptimizerAlgorithm()
algorithm.start_optimizer(dataframe = df_data_metrics_coppied.drop('ÇIKIŞ', axis=1),
                          min_vector=expected_min_vector,
                          epoch=100)
# (self, dataframe, min_vector, epoch, stop_score)

Epoch is done!
Here are the metric vectors that provide the highest scores achieved
The epoch to reach the maximum score first: 11
-------------------- THE OPTIMIZER DATAFRAME --------------------
   epoch                                                         the_metric  \
11    11              [168, 173, 258, 114, 296, 239, 198, 219, 0, 193, 233]   
12    12              [249, 173, 258, 114, 296, 239, 198, 219, 0, 193, 233]   
13    13              [249, 173, 258, 114, 296, 478, 198, 219, 0, 193, 233]   
23    23              [407, 306, 497, 353, 535, 478, 437, 458, 0, 432, 472]   
24    24              [488, 306, 497, 353, 535, 478, 437, 458, 0, 432, 472]   
26    26              [488, 412, 497, 353, 535, 717, 437, 458, 0, 432, 472]   
36    36              [646, 545, 736, 592, 774, 717, 676, 697, 0, 671, 711]   
37    37              [727, 545, 736, 592, 774, 717, 676, 697, 0, 671, 711]   
39    39              [727, 651, 736, 592, 774, 956, 676, 697, 0, 671, 711]   
50    50     